In [2]:
# !pip install http://download.pytorch.org/whl/cu90/torch-1.0.0-cp36-cp36m-linux_x86_64.whl
# !pip install torchvision
# !pip install tqdm

## 손글씨 문자를 판별하는 MLP 작성

In [3]:
import torch
from torch import nn

net = nn.Sequential(
    nn.Linear(64, 32),
    nn.ReLU(),
    nn.Linear(32, 16),
    nn.ReLU(),
    nn.Linear(16, 10)
)

In [4]:
# optimizer 정의
# Loss 함수 정의

from torch import optim
from sklearn.datasets import load_digits
digits = load_digits()

X = digits.data
Y = digits.target

# NumPy의 ndarray를 PyTorch의 Tensor로 변환
X = torch.tensor(X, dtype=torch.float32)
Y = torch.tensor(Y, dtype=torch.int64)

# 소프트맥스 크로스 엔트로피
loss_fn = nn.CrossEntropyLoss()

# Adam
optimizer = optim.Adam(net.parameters())

In [5]:
# 손실 함수의 로그
losses = []

# 100회 반복
for epoc in range(100):
     # backward 메서드로 계산된
    #이전 값을 삭제
    optimizer.zero_grad()

    # 선형 모델로 y의 예측 값 계산
    y_pred = net(X)

    # MSE loss와 w를 사용한 미분 계산
    loss = loss_fn(y_pred, Y)
    loss.backward()

    # 경사를 갱신
    optimizer.step()

    # 수렴 확인을 위해 loss를 기록해둔다
    losses.append(loss.item())

In [7]:
X = X.to('cpu')
Y = Y.to("cpu")
net.to("cpu")

# 이후 처리는 동일하게 optimizer를 설정해서 학습 루프를 돌린다

Sequential(
  (0): Linear(in_features=64, out_features=32, bias=True)
  (1): ReLU()
  (2): Linear(in_features=32, out_features=16, bias=True)
  (3): ReLU()
  (4): Linear(in_features=16, out_features=10, bias=True)
)

In [10]:
from torch.utils.data import TensorDataset, DataLoader


# Dataset 작성
ds = TensorDataset(X, Y)

# 순서로 섞어서 64개씩 데이터를 반환하는 DataLoader 작성
loader = DataLoader(ds, batch_size=64, shuffle=True)

net = nn.Sequential(
    nn.Linear(64, 32),
    nn.ReLU(),
    nn.Linear(32, 16),
    nn.ReLU(),
    nn.Linear(16, 10)
)

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters())

# 최적화 실행
losses = []
for epoch in range(10):
    running_loss = 0.0
    for xx, yy in loader:
        # xx, yy는 64개만는다
        y_pred = net(xx)
        loss = loss_fn(y_pred, yy)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    losses.append(running_loss)
print(losses)

[61.73985683917999, 42.629998207092285, 24.565683364868164, 14.431542694568634, 9.543607011437416, 7.341830536723137, 5.806207753717899, 5.184779979288578, 4.404019859619439, 3.857878103852272]
